## Prerequisites


Make sure you have the following packages installed before running the notebook cells:
 - NumPy
 - Keras
 - Tensorflow
 - pydot
 - graphviz
 - matplotlib

In [1]:
import os
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras import (layers, models, 
                   optimizers)
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [ ]:
SEED = 9
BATCH_SIZE = 64
np.random.seed(SEED)
tf.set_random_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# import subprocess
# subprocess.call(['unzip', '/content/drive/My Drive/Colab Notebooks/new_art.zip', '-d', '/content/drive/My Drive/Colab Notebooks/new_art'])

0

In [ ]:
# import shutil
# shutil.rmtree('/content/depth_data/images')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Normalisation.

In [ ]:
MEAN = "/content/drive/My Drive/Colab Notebooks/depth_data/depth_data/IMG_depth_mean.npz"
STD = "/content/drive/My Drive/Colab Notebooks/depth_data/depth_data/IMG_depth_std.npz"

mean = np.load(MEAN)['mean']
std = np.load(STD)['std']

# mean = np.reshape(mean, (128, 128, 3))
# std = np.reshape(std, (128, 128, 3))

In [ ]:
def unet_generator(gen1, gen2):
    
    while True:
        im_op = next(gen1)
        mask_op = next(gen2)
        
        yield [im_op, mask_op]

In [ ]:
# from preprocessing.image import ImageDataGenerator

def preprocess(x):
    return (x - mean)/(std + K.epsilon())

# For train

# IMP: ENSURE THAT AUGMENTATIONS ARE CONSISTENT!

# Base augmentations
base_augmentations = dict (
    horizontal_flip = True,
    vertical_flip = True,
    rotation_range = 60,
    height_shift_range = 0.1,
    width_shift_range = 0.1
)

# Make a copy.
im_augmentations = dict(base_augmentations)
mask_augmentations = dict(base_augmentations)

# Add the preprocessing step.
im_augmentations["preprocessing_function"] = preprocess
# mask_augmentations["rescale"] = 1./255

im_gen = ImageDataGenerator(**im_augmentations)
mask_gen = ImageDataGenerator(**mask_augmentations)

# -------------------------------------------------

# For val and test
im_gen_no_augment = ImageDataGenerator(preprocessing_function=preprocess)
# mask_gen_no_augment = ImageDataGenerator(rescale=1./255)
mask_gen_no_augment = ImageDataGenerator()

In [ ]:
im_gen

In [ ]:
mask_gen

In [ ]:
# print()
print("Train:")

# ====== Train generators ====== #
im_train_gen = im_gen.flow_from_directory("/content/drive/My Drive/Colab Notebooks/depth_data/depth_data/images/", 
                                          classes=["train"], 
                                          target_size=(128, 128),
                                          seed=SEED, 
                                          batch_size=BATCH_SIZE, 
                                          class_mode=None, 
                                          shuffle=True)

mask_train_gen = mask_gen.flow_from_directory("/content/drive/My Drive/Colab Notebooks/depth_data/depth_data/masks/", 
                                              classes=["train"], 
                                              target_size=(128, 128),
                                              seed=SEED, 
                                              batch_size=BATCH_SIZE, 
                                              class_mode=None,
                                              shuffle=True, 
                                              color_mode='grayscale')

train_gen = unet_generator(im_train_gen, mask_train_gen)

# ====== Validation generators ====== #

print()
print("Validation:")

im_val_gen = im_gen_no_augment.flow_from_directory("/content/drive/My Drive/Colab Notebooks/depth_data/depth_data/images/", 
                                                   classes=["val"], 
                                                   target_size=(128, 128),
                                                   seed=SEED, 
                                                   batch_size=BATCH_SIZE, 
                                                   class_mode=None, 
                                                   shuffle=True)

mask_val_gen = mask_gen_no_augment.flow_from_directory("/content/drive/My Drive/Colab Notebooks/depth_data/depth_data/masks/", 
                                                       classes=["val"], 
                                                       target_size=(128, 128),
                                                       seed=SEED, 
                                                       batch_size=BATCH_SIZE, 
                                                       class_mode=None,
                                                       shuffle=True,
                                                       color_mode='grayscale')

val_gen = unet_generator(im_val_gen, mask_val_gen)

# ====== Test generators ====== #

print()
print("Test:")

im_test_gen = im_gen_no_augment.flow_from_directory("/content/drive/My Drive/Colab Notebooks/depth_data/depth_data/images/", 
                                                    classes=["test"], 
                                                    target_size=(128, 128),
                                                    seed=SEED, 
                                                    batch_size=BATCH_SIZE, 
                                                    class_mode=None, 
                                                    shuffle=True,)

mask_test_gen = mask_gen_no_augment.flow_from_directory("/content/drive/My Drive/Colab Notebooks/depth_data/depth_data/masks/", 
                                                        classes=["test"], 
                                                        target_size=(128, 128),
                                                        seed=SEED, 
                                                        batch_size=BATCH_SIZE, 
                                                        class_mode=None,
                                                        shuffle=True, 
                                                        color_mode='grayscale')

test_gen = unet_generator(im_test_gen, mask_test_gen)

Train:
Found 1599 images belonging to 1 classes.
Found 1599 images belonging to 1 classes.

Validation:
Found 343 images belonging to 1 classes.
Found 343 images belonging to 1 classes.

Test:
Found 342 images belonging to 1 classes.
Found 342 images belonging to 1 classes.


In [ ]:
print(len(next(train_gen)))
print(np.shape(next(train_gen)[0]))
print(np.shape(next(train_gen)[1]))

2
(64, 128, 128, 3)
(64, 128, 128, 1)


In [ ]:
from keras.layers import Dropout
from keras import regularizers

In [ ]:
def UNet():
    _in_ = layers.Input(shape=(128, 128, 3))
    reg = 1e-3
    drop = 0.55
    c=1
    _alpha = 0.1
    # 120, 120, 8
    x = layers.Conv2D(8*c,
                      kernel_size=(3, 3),
                      strides=(1, 1),
                      padding="same",
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(_in_)
    x = layers.BatchNormalization()(x)
    # out1 = layers.Activation("relu", name="out1")(x)
    out1 = layers.LeakyReLU(alpha = _alpha)(x)
    
    # 60, 60, 8
    x = layers.MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2),
                            padding="same")(out1)
    x = layers.BatchNormalization()(x)
#     x = Dropout(drop)(x)
    # x = layers.Activation("relu")(x)
    
    # 60, 60, 16
    x = layers.Conv2D(16*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1),
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    out2 = layers.LeakyReLU(alpha = _alpha)(x)
#     x = Dropout(0.3)(x)
    
    # 30, 30, 16
    x = layers.MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2),
                            padding="same")(out2)
    x = layers.BatchNormalization()(x)
#     x = Dropout(drop)(x)
    # x = layers.Activation("relu")(x)
    
    
    # 30, 30, 32
    x = layers.Conv2D(32*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1),
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    out3 = layers.LeakyReLU(alpha = _alpha)(x)
#     x = Dropout(0.3)(x)
    
    # 15, 15, 32
    x = layers.MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2),
                            padding="same")(out3)
    x = layers.BatchNormalization()(x)
    # x = Dropout(drop)(x)
    # x = layers.Activation("relu")(x)
    
    # 15, 15, 64
    x = layers.Conv2D(64*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1),
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    out4 = layers.LeakyReLU(alpha = _alpha)(x)
#     x = Dropout(0.3)(x)

    # 15, 15, 32
    x = layers.MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2),
                            padding="same")(out4)
    x = layers.BatchNormalization()(x)
    # x = Dropout(drop)(x)
    # x = layers.Activation("relu")(x)
    
    # 15, 15, 64
    x = layers.Conv2D(128*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1),
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    out5 = layers.LeakyReLU(alpha = _alpha)(x)
#     x = Dropout(0.3)(x)

    # 15, 15, 32
    x = layers.MaxPooling2D(pool_size=(2, 2),
                            strides=(2, 2),
                            padding="same")(out5)
    x = layers.BatchNormalization()(x)
    # x = Dropout(drop)(x)
    # x = layers.Activation("relu")(x)
    
    # 15, 15, 64
    x = layers.Conv2D(256*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1),
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
#     x = Dropout(0.3)(x)
    
    # Upscaling begins here.

        # 30, 30, 32
    x = layers.Conv2DTranspose(128*c, 
                               kernel_size=(2, 2), 
                               strides=(2, 2), 
                               padding="same",
                               use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Concatenate(axis=-1)([x, out5])
    # x = Dropout(drop)(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
    
    # 30, 30, 32
    x = layers.Conv2D(128*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1), 
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
#     x = Dropout(0.3)(x)
    
         # 30, 30, 32
    x = layers.Conv2DTranspose(64*c, 
                               kernel_size=(2, 2), 
                               strides=(2, 2), 
                               padding="same",
                               use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Concatenate(axis=-1)([x, out4])
    # x = Dropout(drop)(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
    
    # 30, 30, 32
    x = layers.Conv2D(64*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1), 
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
#     x = Dropout(0.3)(x)
    
    
    
    # 30, 30, 32
    x = layers.Conv2DTranspose(32*c, 
                               kernel_size=(2, 2), 
                               strides=(2, 2), 
                               padding="same",
                               use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Concatenate(axis=-1)([x, out3])
#     x = Dropout(drop)(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
    
    # 30, 30, 32
    x = layers.Conv2D(32*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1), 
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
#     x = Dropout(0.3)(x)
    
    # 60, 60, 16
    x = layers.Conv2DTranspose(16*c, 
                               kernel_size=(2, 2), 
                               strides=(2, 2), 
                               padding="same",
                               use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Concatenate(axis=-1)([x, out2])
    # x = Dropout(drop)(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
    
    # 60, 60, 16
    x = layers.Conv2D(16*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1), 
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)

    
    # 120, 120, 8
    x = layers.Conv2DTranspose(8*c, 
                               kernel_size=(2, 2), 
                               strides=(2, 2), 
                               padding="same",
                               use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Concatenate(axis=-1)([x, out1])
    # x = Dropout(drop)(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)
    
    # 120, 120, 3
    x = layers.Conv2D(3*c, 
                      kernel_size=(3, 3), 
                      strides=(1, 1), 
                      padding="same", 
                      use_bias=False,
                      kernel_regularizer=regularizers.l2(reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha = _alpha)(x)

    
#     120, 120, 1
    x = layers.Conv2D(1, 
                      kernel_size=(1, 1), 
                      strides=(1, 1), 
                      padding="same", 
                      use_bias=False)(x)

    
    _out_ = layers.Activation('relu')(x)
    
    return models.Model(_in_, _out_)

In [ ]:
model = UNet()

In [ ]:
  print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 128, 128, 8)  216         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 128, 128, 8)  32          conv2d_13[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_17 (LeakyReLU)      (None, 128, 128, 8)  0           batch_normalization_22[0][0]     
____________________________________________________________________________________________

In [ ]:
# plot_model(model=model, 
#            to_file="unet2.png", 
#            show_shapes=True, 
#            show_layer_names=True)

In [ ]:
def dice_coefficient(y_true, y_pred):
    
    num_axes = len(K.int_shape(y_true))
    
    axes = list(range(1, num_axes))
    
    numerator = 2. * K.sum(y_true * y_pred, axis=axes)
    denominator = K.sum(y_true, axis=axes) + K.sum(y_pred, axis=axes)
    
    return K.mean(numerator/(denominator + K.epsilon()), axis=0)

def dice_loss(y_true, y_pred):
    return 1. - dice_coefficient(y_true, y_pred)

In [ ]:
def depth_loss_function(y_true, y_pred, theta=0.85, maxDepthVal=1000.0/10.0):
    
    # Point-wise depth
    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

    # Edges
    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)
    l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)

    # Structural similarity (SSIM) index
    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, maxDepthVal)) * 0.5, 0, 1)

    # Weights
    w1 = 0.08
    w2 = 0.07
    w3 = theta

    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))

In [ ]:
from keras.losses import mean_squared_error, categorical_crossentropy
from keras.utils import to_categorical
def cat_loss(y_true, y_pred):
  y_true_cat=to_categorical(y_true)
  y_true = y_true_cat.reshape(y_true.shape[0], y_true.shape[1], y_true_cat.shape[1])
  y_true=tf.argmax(y_true,axis=-1)
  y_pred_cat=to_categorical(y_pred)
  y_pred = y_pred_cat.reshape(y_pred.shape[0], y_pred.shape[1], y_true_cat.shape[1])
  y_pred=tf.argmax(y_pred,axis=-1)
  return K.mean(categorical_crossentropy(y_true, y_pred))

In [ ]:
import keras.losses

In [ ]:
from keras.metrics import mean_squared_error as accuracy
def acc(y_true, y_pred):
    return K.mean(accuracy(y_true, y_pred))

In [ ]:
def combined_loss(y_true, y_pred):
    rms_wt = 0
    depth_wt = 1
    return rms_wt * acc(y_true, y_pred) + depth_wt * depth_loss_function(y_true, y_pred)

In [ ]:
# optim = optimizers.sgd(lr=.1, decay=3e-4, momentum=0.99)
optim = optimizers.adam(lr=7e-2, decay=3e-4)
# optim = optimizers.Adadelta()

model.compile(loss=combined_loss, metrics=[combined_loss], optimizer=optim)

In [ ]:
from keras.callbacks import LearningRateScheduler

decay_interval = 2
decay_intensity = 0.6

def schedule(epoch, lr):
    if epoch > 0 and (epoch+1)%decay_interval==0:
        lr *= decay_intensity
    
    return lr

scheduler = LearningRateScheduler(schedule, verbose=1)

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint_name = "UNet_depth_best_val_loss.h5"

checkpointer = ModelCheckpoint(checkpoint_name,
                               monitor='val_loss', mode='min', 
                               save_best_only=True, 
                               save_weights_only=False, 
                               verbose=1)

In [ ]:
import time

In [ ]:
%%time
# Fit.
num_epochs = 20
train_steps = (im_train_gen.samples//BATCH_SIZE) + (im_train_gen.samples%BATCH_SIZE!=0)
val_steps = (im_val_gen.samples//BATCH_SIZE) + (im_val_gen.samples%BATCH_SIZE!=0)

h = model.fit_generator(train_gen, 
                        steps_per_epoch=train_steps, 
                        validation_data=val_gen, 
                        validation_steps=val_steps, 
                        epochs=num_epochs, 
                        shuffle=True, 
                        verbose=1, 
                        callbacks=[scheduler, checkpointer],
                        use_multiprocessing = True)

print("Done Training")


Epoch 1/20

Epoch 00001: LearningRateScheduler setting learning rate to 0.07000000029802322.
13/25 [==============>...............] - ETA: 25s - loss: 76.9645 - combined_loss: 76.6995

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


25/25 [==============================] - 673s 27s/step - loss: 72.2211 - combined_loss: 71.9662 - val_loss: 76.0952 - val_combined_loss: 75.8898

Epoch 00001: val_loss improved from inf to 76.09524, saving model to UNet_depth_best_val_loss.h5
Epoch 2/20

Epoch 00002: LearningRateScheduler setting learning rate to 0.04200000017881393.
25/25 [==============================] - 13s 507ms/step - loss: 54.3096 - combined_loss: 54.0829 - val_loss: 36.2942 - val_combined_loss: 35.8634

Epoch 00002: val_loss improved from 76.09524 to 36.29421, saving model to UNet_depth_best_val_loss.h5
Epoch 3/20

Epoch 00003: LearningRateScheduler setting learning rate to 0.041999999433755875.
25/25 [==============================] - 20s 807ms/step - loss: 42.2093 - combined_loss: 41.6861 - val_loss: 88.6462 - val_combined_loss: 87.7138

Epoch 00003: val_loss did not improve from 36.29421
Epoch 4/20

Epoch 00004: LearningRateScheduler setting learning rate to 0.025199999660253523.
25/25 [=====================

In [ ]:
# Plot the curves.
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Dice loss.
ax[0].plot(h.history['loss'], 'r-')
ax[0].plot(h.history['val_loss'], 'b-')
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Dice Loss")
ax[0].legend(["Training Dice loss", "Validation Dice loss"])

ax[1].plot(h.history['combined_loss'], 'r-')
ax[1].plot(h.history['val_combined_loss'], 'b-')
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Dice Coefficient")
ax[1].legend(["Training Dice Coefficient", "Validation Dice Coefficient"])

plt.show()

In [ ]:
model.load_weights("/content/UNet_depth_best_val_loss.h5")

In [ ]:
num_outputs = 10

fig, ax = plt.subplots(nrows=num_outputs, ncols=3, figsize=(15, 25))

batch = next(test_gen)
# file_names = next(im_test_gen)

for ix in range(num_outputs):
    # 120, 120, 3
    im = batch[0][ix,:]
    
    # De-normalize image.
    im = np.clip((im * std) + mean, 0, 255).astype(np.uint8)
    
    # 120, 120, 1
    mask = batch[1][ix,:]
    
    # 1, 120, 120, 3
    im_pred = np.expand_dims(im, axis=0)
    
    # 1, 120, 120, 1
    pred = model.predict(im_pred)
    
#     print(pred)
    
    # 120, 120, 3
    pred = np.squeeze(pred, axis=0)
    
    # 120, 120, 1 -> 120, 120
    mask = np.squeeze(mask, axis=-1)
    
    # 120, 120, 1 -> 120, 120
    pred = np.squeeze(pred, axis=-1)
#     pred = ((pred/pred.max())**1.3)
#     pred = ((pred/pred.max())*255)
    
    ax[ix, 0].imshow(im)
    ax[ix, 1].imshow(mask, cmap='gray')
    ax[ix, 2].imshow(pred, cmap='gray')
#     ax[ix, 0].set_xlabel(file_names[ix])

In [ ]:
for i, l in enumerate(model.get_weights()):
  if len(l.shape) > 2:
    print(l.shape, i, l.min(), l.max())

In [ ]:
import cv2
import numpy as np

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
imb = np.random.randint(120, 250, (100,100,100)).astype('uint8')

In [ ]:
def on_trackbar(img_no):
    img = imb[img_no, :, :].astype('uint8')
    img = cv2.putText(img, str(5), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, .4, (0,255,255), 2)
    cv2_imshow("CT Labeled", img)

In [ ]:
trackbar_name = "CT #"
title_window = "CT Labeled"
cv2.namedWindow(title_window)
# cv2.createTrackbar(trackbar_name, title_window , 0, img.shape[0] - 1, on_trackbar)
cv2.createTrackbar(trackbar_name, title_window , 0, 99, on_trackbar)
on_trackbar(0)
while show==True:
        k = cv2.waitKey(1) & 0xFF
        #exit
        if k == ord('x') or k == ord('X'):
            show = False
        if k == ord('q') or k == ord('Q'):
            show = False
            cv2.destroyAllWindows()
cv2.destroyAllWindows()